# 🧠 Bayesian-AI Dashboard

**Objective:** Consolidated interface for System Verification, Debugging, and Full Learning Cycle.
**Sections:**
1. Preflight & Environment Checks (Deep Audit)
2. Data Pipeline Test
3. Core Component Tests
4. Quick Learn (3-Day Simulation)
5. Mini Training Run (5 Iterations)
6. Full Learning Cycle (Production Run)
7. Result Analysis & Visualization
8. Utilities

## 1. Preflight & Environment Checks ✈️
Verify Python environment, CUDA availability, and run Deep Audit.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output
from numba import cuda

# Add root to path
current_dir = Path(os.getcwd())
project_root = current_dir
while not (project_root / 'requirements.txt').exists():
    parent = project_root.parent
    if parent == project_root:
        break
    project_root = parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
print(f"Project Root: {project_root}")

import importlib
import config.settings
importlib.reload(config.settings)
from config.settings import OPERATIONAL_MODE, RAW_DATA_PATH, ANCHOR_DATE
import cuda_modules.hardened_verification as hv
from training.orchestrator import TrainingOrchestrator, load_data_from_directory, get_data_source

# 1. Check Operational Mode
print(f"\nOperational Mode: {OPERATIONAL_MODE}")

# 2. Check CUDA
CUDA_LOCKED = False
try:
    if cuda.is_available():
        print(f"🟢 CUDA Available: {cuda.detect()}")
        device = cuda.get_current_device()
        print(f"   Device: {device.name}")
        CUDA_LOCKED = True
    else:
        print("🔴 CUDA Not Available. System running in CPU mode.")
except Exception as e:
    print(f"🔴 CUDA Check Failed: {e}")

print(f"🔒 CUDA Locked Mode: {CUDA_LOCKED}")

# 3. Deep Audit
print("\n--- Deep Audit ---")
try:
    if hv.run_audit():
        print("🟢 Deep Audit Passed")
    else:
        print("🔴 Deep Audit Failed (See CUDA_Debug.log)")
except Exception as e:
    print(f"🔴 Audit Execution Error: {e}")

## 2. Data Pipeline Test 📊
Validate loading of a single data file.

In [ ]:
from tests.utils import get_test_data_files

try:
    files = get_test_data_files()
    
    # Filter by Anchor Date
    def get_date_from_filename(fname):
        import re
        match = re.search(r"(\d{8})", fname)
        return match.group(1) if match else "00000000"

    anchor_str = ANCHOR_DATE.replace("-", "")
    
    # Sort and Filter
    valid_files = []
    if files:
        # Sort chronologically
        files.sort(key=lambda x: get_date_from_filename(os.path.basename(x)))
        
        # Filter >= Anchor Date
        valid_files = [f for f in files if get_date_from_filename(os.path.basename(f)) >= anchor_str]
        
        if not valid_files:
             print(f"⚠️ No files found after anchor date {ANCHOR_DATE}. Using earliest available.")
             valid_files = files # Fallback
    
    if not valid_files:
        print("🔴 No data files found.")
    else:
        test_file = valid_files[0]
        print(f"Loading {os.path.basename(test_file)} (Anchor: {ANCHOR_DATE})...")
        df = get_data_source(test_file)
        print(f"🟢 Load Success! Shape: {df.shape}")
        
        # Store for later use
        sample_data = df.head(1000).copy() if len(df) > 1000 else df.copy()

except Exception as e:
    print(f"🔴 Pipeline Test Failed: {e}")

## 3. Core Component Tests ⚙️
Verify StateVector, BayesianBrain, LayerEngine.

In [ ]:
from core.state_vector import StateVector
from core.bayesian_brain import BayesianBrain
from core.layer_engine import LayerEngine

print("--- Component Status ---")

# 1. StateVector
try:
    sv = StateVector.null_state()
    assert hash(sv) is not None
    print("🟢 StateVector: Operational")
except Exception as e:
    print(f"🔴 StateVector: Failed ({e})")

# 2. BayesianBrain
try:
    bb = BayesianBrain()
    print("🟢 BayesianBrain: Initialized")
except Exception as e:
    print(f"🔴 BayesianBrain: Failed ({e})")

# 3. LayerEngine
try:
    le = LayerEngine(use_gpu=CUDA_LOCKED)
    print(f"🟢 LayerEngine: Initialized (GPU={le.use_gpu})")
except Exception as e:
    print(f"🔴 LayerEngine: Failed ({e})")

## 4. Quick Learn: 3-Day Simulation 🎲
Randomly select 3 data files and run isolated learning simulations.

In [ ]:
import glob
import random

# UI Helper: Simulation Monitor
class SimulationMonitor:
    def __init__(self, title="Simulation Progress", max_steps=100):
        self.lbl_title = widgets.HTML(f"<b>{title}</b>")
        self.progress = widgets.IntProgress(value=0, min=0, max=max_steps, description='Progress:', bar_style='info')
        self.lbl_status = widgets.Label(value="Initializing...")
        self.log_area = widgets.Output(layout={'border': '1px solid #ccc', 'height': '200px', 'overflow_y': 'scroll'})
        self.container = widgets.VBox(
            [self.lbl_title, self.progress, self.lbl_status, self.log_area],
            layout=widgets.Layout(border='2px solid #007bff', padding='10px', margin='10px 0')
        )
        
    def show(self):
        display(self.container)
        
    def log(self, message):
        with self.log_area:
            print(message)
            
    def update_progress(self, value, status=None):
        self.progress.value = value
        if status:
            self.lbl_status.value = status
            
    def complete(self, success=True):
        self.progress.value = self.progress.max
        self.progress.bar_style = 'success' if success else 'danger'
        self.lbl_status.value = "Complete" if success else "Failed"

def run_quick_learn(monitor, verbose=False):
    # Find data files
    monitor.log("Scanning for data files...")
    files = glob.glob(str(project_root / 'DATA/RAW/*.parquet')) + glob.glob(str(project_root / 'DATA/RAW/*.dbn*'))
    
    if not files:
        monitor.log("🔴 No data files found in DATA/RAW")
        monitor.complete(False)
        return

    # Filter by Anchor Date
    def get_date_from_filename(fname):
        import re
        match = re.search(r"(\d{8})", fname)
        return match.group(1) if match else "00000000"

    anchor_str = ANCHOR_DATE.replace("-", "")
    
    # Sort chronologically
    files.sort(key=lambda x: get_date_from_filename(os.path.basename(x)))
    
    # Filter
    valid_files = [f for f in files if get_date_from_filename(os.path.basename(f)) >= anchor_str]
    
    if not valid_files:
         monitor.log(f"⚠️ No files found after {ANCHOR_DATE}. Using first 3 available.")
         valid_files = files
    
    # Select first 3 files (Deterministic)
    selected_files = valid_files[:3]
    monitor.log(f"Selected {len(selected_files)} files (Anchor {ANCHOR_DATE}): {[os.path.basename(f) for f in selected_files]}")
    
    monitor.progress.max = len(selected_files)
    results = []
    
    for i, file_path in enumerate(selected_files):
        monitor.update_progress(i, f"Processing {os.path.basename(file_path)}...")
        monitor.log(f"--- Loading {os.path.basename(file_path)} ---")
        
        try:
            # Load Single File
            df = get_data_source(file_path)
            monitor.log(f"Loaded {len(df)} ticks.")
            
            # Initialize Orchestrator
            orch = TrainingOrchestrator(
                asset_ticker='MNQ', 
                data=df, 
                output_dir='debug_outputs/quick_learn',
                use_gpu=CUDA_LOCKED,
                verbose=verbose
            )
            
            # Run 1 iteration
            metrics = orch.run_training(iterations=1, params={'mode': 'quick_learn'})
            metrics['file'] = os.path.basename(file_path)
            results.append(metrics)
            monitor.log(f"Result: PnL=${metrics['pnl']:.2f}, WR={metrics['win_rate']:.1%}")
            
        except Exception as e:
            monitor.log(f"🔴 Error processing file: {e}")
            
    monitor.complete(True)
    
    # Summary
    if results:
        print("\n=== Quick Learn Summary ===")
        res_df = pd.DataFrame(results)
        display(res_df[['file', 'total_trades', 'win_rate', 'pnl']])

chk_verbose = widgets.Checkbox(value=False, description='Verbose Mode')
btn_quick = widgets.Button(description="Run 3-File Sim", button_style='info')

def on_quick_click(b):
    mon = SimulationMonitor("Quick Learn Simulation")
    mon.show()
    run_quick_learn(mon, verbose=chk_verbose.value)

btn_quick.on_click(on_quick_click)
display(widgets.HBox([btn_quick, chk_verbose]))

## 5. Mini Training Run (5 Iterations) 🏃‍♂️
Interactive 5-iteration training on sample data (loaded in Sec 2).

In [ ]:
def run_mini_training(b):
    with out_area:
        clear_output()
        print("Initializing Mini Training (5 iterations)...")
        
        if 'sample_data' not in globals():
            print("🔴 Sample data not loaded. Run Section 2 first.")
            return

        try:
            orch = TrainingOrchestrator(
                asset_ticker='MNQ', 
                data=sample_data, 
                output_dir='debug_outputs/mini_run',
                use_gpu=CUDA_LOCKED,
                verbose=chk_verbose_mini.value
            )
            
            res = orch.run_training(iterations=5)
            print("\n🟢 Mini Run Complete!")
            print(f"Total Trades: {res['total_trades']}")
            print(f"PnL: ${res['pnl']:.2f}")
            print(f"Win Rate: {res['win_rate']:.1%}")
            
        except Exception as e:
            print(f"🔴 Training Failed: {e}")

chk_verbose_mini = widgets.Checkbox(value=False, description='Verbose Mode')
btn_run = widgets.Button(description="Run 5 Iterations", button_style='primary')
out_area = widgets.Output()

btn_run.on_click(run_mini_training)
display(widgets.VBox([widgets.HBox([btn_run, chk_verbose_mini]), out_area]))

## 6. Full Learning Cycle (Production Run) 🚀
Loads all available data and executes the full training loop with real-time visualization.

In [ ]:
# Setup Widgets for Full Run
fig_full = go.FigureWidget()
fig_full.add_scatter(name="Daily PnL", y=[], mode='lines+markers', line=dict(color='green'))
fig_full.add_scatter(name="Avg Confidence", y=[], mode='lines', line=dict(color='blue', dash='dot'), yaxis='y2')

fig_full.update_layout(
    title="Learning Progress: PnL & Confidence",
    xaxis_title="Iteration",
    yaxis_title="PnL ($)",
    yaxis2=dict(
        title="Avg Confidence",
        overlaying='y',
        side='right',
        range=[0, 1]
    ),
    height=500,
    template="plotly_dark"
)

progress_bar_full = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    style={'bar_color': '#00ff00'},
    layout=widgets.Layout(width='100%')
)

status_label_full = widgets.Label(value="Ready to start full training.")
stats_output_full = widgets.Output()

def update_dashboard_full(metrics):
    # Update Charts
    with fig_full.batch_update():
        fig_full.data[0].y = list(fig_full.data[0].y) + [metrics['pnl']]
        fig_full.data[1].y = list(fig_full.data[1].y) + [metrics['average_confidence']]
        fig_full.data[0].x = list(range(1, len(fig_full.data[0].y) + 1))
        fig_full.data[1].x = list(range(1, len(fig_full.data[1].y) + 1))

    # Update Progress
    progress_bar_full.value = metrics['iteration']
    progress_bar_full.max = metrics['total_iterations']
    
    status_label_full.value = f"Iter {metrics['iteration']}/{metrics['total_iterations']} | PnL: ${metrics['pnl']:.2f} | WR: {metrics['win_rate']:.1%} | Conf: {metrics['average_confidence']:.2f}"

ITERATIONS_DEFAULT = 50
btn_start_full = widgets.Button(description="Start Full Training", button_style='success', icon='play')

def start_full_training(b):
    btn_start_full.disabled = True
    status_label_full.value = "Loading All Data..."
    with stats_output_full:
        clear_output()
    
    try:
        # Check if DATA/RAW exists
        data_path = os.path.join(project_root, 'DATA', 'RAW')
        if not os.path.exists(data_path):
             # Fallback
             data_path = 'DATA/RAW'
        
        print(f"Loading data from: {data_path}...")
        file_paths = load_data_from_directory(data_path)
        dfs = []
        for fp in file_paths:
            try:
                dfs.append(get_data_source(fp))
            except Exception as e:
                print(f"Skipping {fp}: {e}")
        
        if not dfs:
            raise ValueError("No valid data files loaded")
             
        full_data = pd.concat(dfs, ignore_index=True)
        print(f"🟢 Data Loaded. Rows: {len(full_data):,}")
        
        status_label_full.value = "Initializing Orchestrator..."
        
        # Clear previous chart data
        with fig_full.batch_update():
            fig_full.data[0].y = []
            fig_full.data[1].y = []
        
        # Initialize Orchestrator
        orch = TrainingOrchestrator(
            asset_ticker='MNQ', 
            data=full_data, 
            output_dir='models/production_learning',
            use_gpu=CUDA_LOCKED, # Use detected status
            verbose=False
        )
        
        status_label_full.value = "Training Started..."
        
        # Run
        orch.run_training(iterations=ITERATIONS_DEFAULT, on_progress=update_dashboard_full)
        
        status_label_full.value = "✅ Training Complete! Model Saved to models/production_learning/"
        progress_bar_full.bar_style = 'success'
        
    except Exception as e:
        status_label_full.value = f"❌ Error: {str(e)}"
        progress_bar_full.bar_style = 'danger'
        with stats_output_full:
            print(f"Training Exception: {e}")
            import traceback
            traceback.print_exc()
    finally:
        btn_start_full.disabled = False

btn_start_full.on_click(start_full_training)
display(widgets.VBox([btn_start_full, progress_bar_full, status_label_full, fig_full, stats_output_full]))

## 7. Result Analysis & Visualization 📈
Analyze the learned probability tables and visualize performance.

In [ ]:
import pickle

def inspect_table(path, title):
    if not os.path.exists(path):
        print(f"⚪ {title}: File not found ({path})")
        return
    
    print(f"Loading {path}...")
    try:
        with open(path, 'rb') as f:
            data = pickle.load(f)
        
        # Handle both raw dict and BayesianBrain object
        table = data['table'] if isinstance(data, dict) and 'table' in data else getattr(data, 'table', {})
        
        print(f"🟢 {title}: {len(table)} states learned.")
        
        # Convert to DF
        records = []
        for state, stats in table.items():
            total = stats['total']
            wins = stats['wins']
            if total > 0:
                records.append({
                    'total': total,
                    'wins': wins,
                    'win_rate': wins/total,
                    'L1': state.L1_bias,
                    'L5': state.L5_trend
                })
        
        if records:
            df_stats = pd.DataFrame(records)
            fig = px.scatter(df_stats, x='total', y='win_rate', 
                             title=f'{title}: Win Rate vs Sample Size',
                             hover_data=['L1', 'L5'])
            fig.show()
            
    except Exception as e:
        print(f"Error analyzing table: {e}")

btn_analyze = widgets.Button(description="Analyze Models")
btn_analyze.on_click(lambda b: (
    inspect_table('debug_outputs/quick_learn/probability_table.pkl', "Quick Learn Temp"),
    inspect_table('models/probability_table.pkl', "Main Model"),
    inspect_table('models/production_learning/probability_table.pkl', "Production Model"),
    inspect_table('debug_outputs/mini_run/probability_table.pkl', "Mini Run")
))
display(btn_analyze)

In [ ]:
# Visualization Module Integration
print("\n--- Visualization Module Integration ---")
try:
    from visualization.visualization_module import plot_training_results
    
    # Dropdown to select model to visualize
    model_options = [
        ('Production Model', 'models/production_learning/probability_table.pkl'),
        ('Main Model', 'models/probability_table.pkl'),
        ('Quick Learn', 'debug_outputs/quick_learn/probability_table.pkl'),
        ('Mini Run', 'debug_outputs/mini_run/probability_table.pkl')
    ]
    
    dropdown_model = widgets.Dropdown(
        options=model_options,
        description='Select Model:',
    )
    
    btn_viz = widgets.Button(description="Visualize Results", button_style='info')
    
    def on_viz_click(b):
        path = dropdown_model.value
        if os.path.exists(path):
            print(f"Visualizing: {path}")
            # Note: plot_training_results uses plt.show(), which works inline in notebooks
            plot_training_results(path)
        else:
            print(f"🔴 Model file not found: {path}")
            
    btn_viz.on_click(on_viz_click)
    display(widgets.HBox([dropdown_model, btn_viz]))
        
except ImportError:
    print("🔴 Visualization module not found. Check visualization/visualization_module.py")
except Exception as e:
    print(f"🔴 Visualization Integration Error: {e}")

## 8. Utilities 🛠️

In [ ]:
import shutil

def clean_pycache():
    print("Cleaning __pycache__...")
    count = 0
    for root, dirs, files in os.walk('.'):
        for d in dirs:
            if d == '__pycache__':
                shutil.rmtree(os.path.join(root, d))
                count += 1
    print(f"Removed {count} directories.")

btn_clean = widgets.Button(description="Clear PyCache")
btn_clean.on_click(lambda b: clean_pycache())
display(btn_clean)